# Pathos＋global変数の取り扱い
## うまくglobals()は渡せない
- with Pool(os.cpu_count()) as p:


In [2]:
import os
from multiprocessing import Pool, cpu_count 
# from pathos.pools import ParallelPool
from pathos.pools import ProcessPool

from tqdm import tqdm 
from glob import glob
import numpy as np

import dill
import pickle
import pathos_global_test
from test_globals_class import test_global
from pathos_global_test import PathosTest
pathos_test = PathosTest()
top_results = [[0, 0] for _ in range(11)]
pathos_test.top_namespace = globals()
print(f'global namespace = {pathos_test.top_namespace}')

# initializerの定義
# https://superfastpython.com/multiprocessing-pool-shared-global-variables/
def init_worker(globals_object):
    print(f'globals object = {globals_object}')
    pathos_test.top_namespace = globals_object
globals_test = test_global()
a = 0
b = 1
c = 2

globals_test.add('a','b','c',globals())

# wrap_save_npy.namespace_main = globals()
# print(wrap_save_npy.namespace_main)
# サブプロセス群に入力データを分配 (データ並列) して関数を並列実行する
# p = Pool(processes=cpu_count()-1)
# pool = ParallelPool(initializer= init_worker, initargs=(globals(),), node=cpu_count()-1)
# pool = ParallelPool( node=cpu_count()-1)
pool = ProcessPool( node=cpu_count()-1)
# NumPy Arrayをsaveするフォルダ
save_base_path = 'gaussian_filtered_nparrays/'
# フォルダ作成
if not os.path.exists(save_base_path):
    os.makedirs(save_base_path)
# pngファイル名リストを作成
path_list = glob('icopng/*.png')

print(f'globals = {pathos_test.top_namespace}')
# processに渡す引数の作成

job_args = [(path, path.split('.')[1], num, 'top_results') for num,path in enumerate(path_list)]
print(job_args[0])
# globals()は動作している
# pathos_test.wrap_save_npy(job_args[0])


# imapによってprocessの順番は任意で処理される

# for result in p.imap_unordered(wrap_save_npy, job_args):
# for result in p.imap(wrap_save_npy, job_args):
# mapでシリアライズされるとglobals()を渡しても動作しなくなる
for result in pool.map(pathos_test.wrap_save_npy, job_args):
    print(result)
# print(f' top results = {top_results}')

global namespace = {'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', "import os\nfrom multiprocessing import Pool, cpu_count \n# from pathos.pools import ParallelPool\nfrom pathos.pools import ProcessPool\n\nfrom tqdm import tqdm \nfrom glob import glob\nimport numpy as np\n\nimport dill\nimport pickle\nimport pathos_global_test\nfrom test_globals_class import test_global\nfrom pathos_global_test import PathosTest\npathos_test = PathosTest()\ntop_results = [[0, 0] for _ in range(11)]\npathos_test.top_namespace = globals()\nprint(f'global namespace = {pathos_test.top_namespace}')\n\n# initializerの定義\n# https://superfastpython.com/multiprocessing-pool-shared-global-variables/\ndef init_worker(globals_object):\n    print(f'globals object = {globals_object}')\n    pathos_te

KeyError: 'top_results'